In [13]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df = pd.read_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 최종정리/first_cleaned_data.csv' )

In [3]:
df

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4,5,5,4,5,5,5,4,4,5,...,5,5,4,5,4,5,5,3,4,5
1,3,1,3,2,3,3,2,1,1,1,...,1,4,4,4,3,5,4,2,5,3
2,2,3,4,2,3,4,1,3,2,1,...,5,5,2,5,4,4,5,3,4,4
3,2,4,2,3,4,4,2,2,1,2,...,4,4,5,4,3,5,4,4,3,3
4,3,3,3,3,5,3,3,1,3,2,...,5,5,5,5,5,5,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813221,4,4,4,3,4,3,3,3,3,3,...,2,4,4,3,4,4,4,2,2,4
813222,4,3,4,3,3,3,4,2,3,3,...,4,5,5,5,5,5,3,4,5,4
813223,4,4,4,3,5,5,4,4,4,2,...,5,5,5,5,4,5,5,5,4,5
813224,2,2,3,2,2,4,1,2,2,2,...,5,4,4,4,3,4,4,5,5,3


In [6]:
# 요인별 문항 리스트 정의
factors = {
    "factor1": [f"EXT{i}" for i in range(1, 11)],
    "factor2": [f"EST{i}" for i in range(1, 11)],
    "factor3": [f"AGR{i}" for i in range(1, 11)],
    "factor4": [f"CSN{i}" for i in range(1, 11)],
    "factor5": [f"OPN{i}" for i in range(1, 11)],
}

In [14]:
scaler = StandardScaler()

In [7]:

# item-total correlation 함수
def get_reduced_index(df_subset):
    high_index = df_subset.corr(method='pearson').sum(axis= 1).sort_values(ascending=False)[:6].index
    return high_index
    # 여기서 상관계수가 가장 높은 상위 5개 index가 뽑혀 나옴.

# 크론바흐 알파 계산
def cronbach_alpha(df_subset):
    item_vars = df_subset.var(axis=0, ddof=1)
    total_var = df_subset.sum(axis=1).var(ddof=1)
    n_items = df_subset.shape[1]
    return (n_items / (n_items - 1)) * (1 - item_vars.sum() / total_var)

In [35]:
# 문항단축 수행 (각 요인별로 상위 6개만 선택)
alpha_afters = []
df_list = []
reduced_items_index = []
for factor_name, items in factors.items():
    df_factor = df[items]
    X_scaled = scaler.fit_transform(df_factor)
    pca = PCA()
    pca.fit(X_scaled)

    #적재값 구하기
    loadings = pd.DataFrame(pca.components_.T,
                            columns=[f'PC{i+1}' for i in range(len(df_factor.columns))],
                            index=df_factor.columns)

    # 예: PC1에서 상위 5개 문항 추출
    top_items = loadings['PC1'].abs().sort_values(ascending=False).head(6)
    for i in top_items.index.values:
        reduced_items_index.append(i)
print(reduced_items_index)


['EXT4', 'EXT5', 'EXT7', 'EXT2', 'EXT10', 'EXT1', 'EST8', 'EST6', 'EST1', 'EST7', 'EST9', 'EST10', 'AGR4', 'AGR9', 'AGR7', 'AGR5', 'AGR2', 'AGR8', 'CSN6', 'CSN1', 'CSN5', 'CSN4', 'CSN9', 'CSN2', 'OPN10', 'OPN2', 'OPN5', 'OPN1', 'OPN3', 'OPN6']


In [39]:
pca_reduced_df = df[reduced_items_index]
pca_reduced_df.to_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 최종정리/Second_pca_reduced_df.csv', index=False)